In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

!pip install transformers
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
import json

import pandas as pd  

     |████████████████████████████████| 4.0 MB 25.0 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 880 kB 72.0 MB/s 
     |████████████████████████████████| 6.6 MB 63.0 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=3606969914e192710eaccc01fda3a3e31234b5d15e98e05109d40e9d86bdf8a1
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|BOS|>', eos_token='<|EOS|>', pad_token='<|pad|>') #gpt2-medium


class GPT2Dataset(Dataset):
  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    for txt in txt_list:
      encodings_dict = tokenizer('<|BOS|>'+ txt + '<|EOS|>', truncation=True, max_length=max_length, padding="max_length")
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# import pandas as pd
# xx = pd.read_csv('summary.csv', encoding = "ISO-8859-1")
# print(xx)

file = 'books_dataset.json'
with open(file) as train_file:
    dict_train = json.load(train_file)

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=True)

xx=train
# data = json.loads(books_dataset)
# df = pd.json_normalize(data['results'])
# xx = df
#print(xx.iloc[:,1])


In [7]:


summaries = xx.iloc[:,1].copy()

print(summaries)

0      <|BOS|>The Time Machine<|SEP|>time faints his ...
1      <|BOS|>The War of the Worlds<|SEP|>envious eye...
2      <|BOS|>A Princess of Mars<|SEP|>lower atmosphe...
3      <|BOS|>Youth<|SEP|>space aliens tiny dead iden...
4      <|BOS|>2 B R 0 2 B<|SEP|>population control ac...
                             ...                        
280    <|BOS|>Rip Foster Rides the Gray Planet<|SEP|>...
281    <|BOS|>Eastern Standard Tribe<|SEP|>members ps...
282    <|BOS|>Man of Many Minds<|SEP|>minds a secret ...
283    <|BOS|>The Players<|SEP|>aliens not nonsense t...
284    <|BOS|>Rip Foster in Ride the Gray Planet<|SEP...
Name: 0, Length: 285, dtype: object


In [8]:
dataset = GPT2Dataset(summaries, tokenizer, max_length=768)
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [18]:
def preprocess_val_dataset(dataset):
  
  s = tokenizer.decode(dataset[0][0]).split('<|SEP|>')
  new_output = s[len(s)-1]
  new_output = s[len(s)-1]
  new_input = s[0]
  for i in range(1,len(s)-1):
    new_input = new_input+'<|SEP|>'+s[i]

  new_input = new_input + '<|SEP|>'
  #print(new_input)
  input_ids = tokenizer.encode(new_input, return_tensors='pt')
  output_ids = tokenizer.encode(new_output, return_tensors='pt')

  return input_ids[0], torch.ones(len(input_ids[0])), output_ids, new_output, new_input

In [19]:
batch_size=2
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [20]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))


Embedding(50260, 768)

In [21]:
epochs = 5
learning_rate = 10e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [22]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [23]:
import time
import datetime
import random
import tensorflow as tf

!pip install ipdb
device = torch.device("cuda")
model.cuda()
total_t0 = time.time()
model = model.to(device)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))
training_stats = []

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        # cc = batch.copy()
        # labels = []
        # for en, ex in enumerate(cc[0]):
        #   import ipdb;ipdb.set_trace()
        #   ind_29 = (ex == 29).nonzero(as_tuple=True)
        #   ind_27 = (ex == 27).nonzero(as_tuple=True)
        #   batch[0][en] = cc[0][en][ind_29[0][1]+1:ind_27[0][2]]
        #   batch[1][en] = cc[1][en][ind_29[0][1]+1:ind_27[0][2]]
        #   labels.append(cc[0][en][ind_29[0][2]+1:])
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    '''
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        input_ids, masks, output_ids, output = preprocess_val_dataset(batch)
        #b_input_ids = batch[0].to(device)
        b_input_ids = input_ids.to(device)
        #b_labels = batch[0].to(device)
        b_labels = input_ids.to(device)
        #b_masks = batch[1].to(device)
        b_masks = masks.to(device)
        #print(f"DEBA: {b_labels}")
        #print(tokenizer.decode(b_input_ids))
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        
    '''
    total_eval_loss = total_train_loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...

  Average training loss: 7.12
  Training epoch took: 0:00:52
  Validation Loss: 16.56
  Validation took: 0:00:52

======== Epoch 2 / 5 ========
Training...

  Average training loss: 2.05
  Training epoch took: 0:00:56
  Validation Loss: 4.76
  Validation took: 0:00:56

======== Epoch 3 / 5 ========
Training...

  Average training loss: 1.56
  Training epoch took: 0:00:55
  Validation Loss: 3.62
  Validation took: 0:00:55

======== Epoch 4 / 5 ========
Training...

  Average training loss: 1.22
  Training epoch took: 0:00:56
  Validation Loss: 2.83
  Validation took: 0:00:56

======== Epoch 5 / 5 ========
Training...

  Average training loss: 1.00
  Training epoch took: 0:00:56
  Validation Loss: 2.32
  Validation took: 0:00:56

Training complete!
Total training took 0:04:35 (h:mm:ss)


In [24]:
model.eval()

prompt = "<|BOS|>we live in solar system<|SEP|> It has nine planets <|SEP|>"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                no_repeat_ngram_size=2,  
                                top_k=5, 
                                max_length = 1000,
                                top_p=0.9, 
                                num_return_sequences=3
                                )
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,   732,  2107,   287,  6591,  1080,    27,    91,  5188,    47,
            91,    29,   632,   468,  5193, 14705,  1279,    91,  5188,    47,
            91,    29]], device='cuda:0')
0: we live in solar system<|SEP|> It has nine planets <|SEP|>The story begins with the discovery of a planet, Helium, which is located in the Helian system. Heliators around the world discover that Helia is home to a civilisation of "gigantic green-skinned, blue-robed, red-necks", and that its inhabitants are intelligent and technologically advanced. The story ends with Helios, a newly discovered planet.


 n n n terrestre stars ter star your served dishes meal And adults hell Sun, sun planets Planet Year The Year stars Sun Planet Year Sun Planet name Planet Born Planet year

 George Washington George "Wellington" West, Jr.
 Seven planets in Heliconia

, named "Little Hel" by the locals
The sun is a starless world with a sunlike "darkened" atmosphere. Its interior is made of thick, frozen 

In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score
from rouge_score import rouge_scorer


for batch in validation_dataloader:
    input_ids, masks, output_ids, ref_story, prompt = preprocess_val_dataset(batch)
    #b_input_ids = batch[0].to(device)
    b_input_ids = input_ids.to(device)
    #b_labels = batch[0].to(device)
    b_labels = input_ids.to(device)
    #b_masks = batch[1].to(device)
    b_masks = masks.to(device)
    #print(f"DEBA: {b_labels}")
    #print(tokenizer.decode(b_input_ids))
  
    #prompt = input_ids

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    print(generated)

    sample_outputs = model.generate(
                                    generated, 
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    no_repeat_ngram_size=2,  
                                    top_k=5, 
                                    max_length = 1000,
                                    top_p=0.9, 
                                    num_return_sequences=3
                                    )
    
    for i, sample_output in enumerate(sample_outputs):
      s =  tokenizer.decode(sample_output, skip_special_tokens=True)
      print(s)
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      scores = scorer.score(s, ref_story)
      print(scores)
      print("************************************")

          
      

ERROR: Invalid requirement: 'rouge/requirements.txt'
Hint: It looks like a path. File 'rouge/requirements.txt' does not exist.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[50257,   220, 50257,  1810, 10572,   286,  8706,    27,    91,  5188,
            47,    91,    29, 36022,  1879,  1010, 12525,  3656, 21303,  2093,
         27487,  1318,  8499,  1545,   262,   911, 27725,  2581, 39439,  3338,
          4398,  8791,   922,  6266,  5093, 13559,  1451,    27,    91,  5188,
            47,    91,    29]], device='cuda:0')
  Warlord of Mars<|SEP|>yellow Carters ally wife unmask disguise Thereafter friend the Shangs request fleets safe haven encounter good orders north polar cap<|SEP|>The novel concerns a war between the Green Men and the "Bad Bloods" (actually the White Walkers) who occupied the planet Issus in the decades-long conflict that occupied their world.
The plot builds tension between Issuance (Green) and his counterpart Valeria, a ruthless and ruthless client-apprentice of the Khylem (a clan of green men). Issuing a peace treaty with Issues, Issures the mercantile Issgard, who has trained him to be a tactician, and persuades him that h

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.46455223880597013, recall=0.3221216041397154, fmeasure=0.3804430863254393), 'rougeL': Score(precision=0.17164179104477612, recall=0.11901681759379043, fmeasure=0.1405653170359053)}
************************************
  Warlord of Mars<|SEP|>yellow Carters ally wife unmask disguise Thereafter friend the Shangs request fleets safe haven encounter good orders north polar cap<|SEP|>During the year 1866, ships of several nations spot a band of renegade Picts, who have crossed into the unknown south Pacific Ocean. The fleet is attacked and the Pict fleet takes a stand. Captain Nemo is the first to warn the allies.
The Shang brothers, led by seafaring merchant ships, are preparing to attack the fleet and steal supplies from the merchant. Nemos motivation is implied to be both a desire for revenge upon the previous hijacking of the Golden Hornet, and a hatred for the new Americans. While visiting the Barbicane Islands, Nemotsuhito, the son of a merchantman and mas

...

{'rouge1': Score(precision=0.09170305676855896, recall=0.08502024291497975, fmeasure=0.08823529411764705), 'rougeL': Score(precision=0.046579330422125184, recall=0.043184885290148446, fmeasure=0.04481792717086834)}
************************************
tensor([[50257,   220, 50257,  8865,  3827, 34787,    27,    91,  5188,    47,
            91,    29, 42276,   262,  1402, 33486,  2779,  5473,   257,  7348,
          1221,  2800,  1468,  9846, 42281, 29663,  1056, 22818,  1489, 26730,
         12840, 10441,    12,  2339, 34870,   911,  1236,   447,   247,    82,
            27,    91,  5188,    47,    91,    29]], device='cuda:0')
  Storm Over Warlock<|SEP|>crew the small Terran base refuge a flying disc contact old memories downed scoutship explode booby trap dragon-like humanoid Shann’s<|SEP|>A group of explorers, led by the enigmatic Dr. Goodwin, investigate a mysterious dead trail leading to a vast expansive plain, which some suggest to be a giant, red giant narwhal.
The group disco

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.5166374781085814, recall=0.38612565445026176, fmeasure=0.4419475655430711), 'rougeL': Score(precision=0.18914185639229422, recall=0.14136125654450263, fmeasure=0.16179775280898878)}
************************************
tensor([[50257,   220, 50257,  5498, 39143,    27,    91,  5188,    47,    91,
            29,   305, 13645, 13767,   370,   633,  2277,  2972,  1180, 15807,
         35786, 28421,   262,   649,   530, 16359, 22153,  7448,  5387,  6147,
          4522, 16759,  7386,  4645,    27,    91,  5188,    47,    91,    29]],
       device='cuda:0')
  Second Variety<|SEP|>robot sophisticated Wound hit various different fortune nicknamed claws the new one automated Underground saved internal metal ID dishes domain structure<|SEP|>robots learn to spot and locate a variety of robots, mostly from outside the city. Intelligent robots have a habit of recognizing and recognizing unfamiliar faces, and recognize people with whom it converses.
The robots learn q

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge1': Score(precision=0.45077720207253885, recall=0.33547557840616965, fmeasure=0.3846720707442889), 'rougeL': Score(precision=0.16062176165803108, recall=0.11953727506426735, fmeasure=0.13706705969049374)}
************************************
tensor([[50257,   220, 50257, 17917,    25, 48278,    27,    91,  5188,    47,
            91,    29, 11265,  5384,  2622, 18423,    82,   257,  1762, 13522,
          3708,  9314,   262,  1080,  6591,   649,  2761,   383, 35113,    12,
          2339, 16269,   753, 11970,  6821,  3612,    27,    91,  5188,    47,
            91,    29]], device='cuda:0')
